In [4]:
###############################
############################### Test Code
###############################
import numpy as np
import cv2
import pickle
from tensorflow.keras.models import load_model

In [5]:
#############################################
frameWidth = 640  # CAMERA RESOLUTION
frameHeight = 480
brightness = 180
threshold = 0.75  # PROBABILITY THRESHOLD
font = cv2.FONT_HERSHEY_SIMPLEX
##############################################

In [6]:
# SETUP THE VIDEO CAMERA
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, brightness)

# # IMPORT THE TRAINED MODEL
# pickle_in = open("model_trained.p", "rb")  ## rb = READ BYTE
# model = pickle.load(pickle_in)
model = load_model("model_trained.h5")

In [23]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img


def getCalssName(classNo):
    # Your class names mapping function here
    pass

In [24]:
while True:

    # READ IMAGE
    success, imgOrignal = cap.read()

    # PROCESS IMAGE
    img = np.asarray(imgOrignal)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)
    cv2.imshow("Processed Image", img)
    img = img.reshape(1, 32, 32, 1)
    cv2.putText(
        imgOrignal, "CLASS: ", (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA
    )
    cv2.putText(
        imgOrignal, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA
    )
    # PREDICT IMAGE
    predictions = model.predict(img)
    classIndex = np.argmax(predictions)
    probabilityValue = np.amax(predictions)
    if probabilityValue > threshold:
        cv2.putText(
            imgOrignal,
            str(classIndex) + " " + str(getCalssName(classIndex)),
            (120, 35),
            font,
            0.75,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
        cv2.putText(
            imgOrignal,
            str(round(probabilityValue * 100, 2)) + "%",
            (180, 75),
            font,
            0.75,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
    cv2.imshow("Result", imgOrignal)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()